In [1]:
## required packages
import numpy as np
# import matplotlib.pyplot as plt
# import os, time
import importlib
import im.fcn_im as fcn_im
import pandas as pd
# from numpy.random import standard_normal
# from scipy.linalg import cholesky
import model, fcn_gen, os, time
from scipy import sparse
# import OpenSHAInterface as sha

In [2]:
# base inputs
site_loc_file = r'C:\Users\barry\Desktop\OpenSHAInterface\CA_pipe_midpt_allsites.txt'
rate_min = 1/10000
im_tool = 'sha'

im_dir = r'C:\Users\barry\Desktop\OpenSHAInterface\im_output_10000yr_allsites'
dm_dir = r'C:\Users\barry\Desktop\OpenSHAInterface\rr_out'
rup_meta_file = r'C:\Users\barry\Desktop\OpenSHAInterface\rup_meta_10000yr_allsites.hdf5'
# sha_meta_file = r'C:\Users\barry\Desktop\OpenSHAInterface\sha_meta_10000yr.txt'
groups2run = np.loadtxt(r'C:\Users\barry\Desktop\OpenSHAInterface\groups2run_allsites.txt',dtype=str)
group_num = 0
num_rups = 2
im_list = ['pgv']

## define number of realizations and generate samples
nsamp=10
flag_corr_d = False
flag_corr_T = False
# geo_cond = 2 #  for spatial correlation

len(groups2run)

1977

In [3]:
##
list_coord_marc = np.asarray([[-121.959102,37.952594],
                  [-121.969138,37.944882],
                  [-121.66069,38.012081],
                  [-121.660565,38.020377],
                  [-121.508865,37.973593],
                  [-121.507891,37.983513],
                  [-121.269766,38.128762],
                  [-121.26935,38.135931]])
ind_marc = [41305, 41312, 29515, 40960, 38581, 76484, 530, 523]
rup_marc = [10, 8, 62]
samp_marc = 5

In [7]:
## reload model.py if it has been modified
importlib.reload(model)
## create class object
A = model.assessment()

In [8]:
count = 0
inc1 = 5
inc2 = 100
# out = {}
time_start = time.time()

In [9]:
## for generating samples
multi = 0
range_start = 0+100*multi
range_end = range_start+1
# range_end = range_start+1
flag_sample_exist = True
for im_group in groups2run[range_start:range_end]:
    #####################################################################
    count += 1
    path_sample = im_dir+'/'+im_group+'/'
    
    #####################################################################
    ## load GM predictions and create random variable
    A.create_RV(im_tool, site_loc_file, im_dir, im_list, rup_meta_file, flag_clear_dict=True, 
                flag_sample_exist=flag_sample_exist, im_group=im_group, num_rups=num_rups)

    #####################################################################
    ## generate/import samples
    A.get_IM(nsamp, im_list, flag_corr_d, flag_corr_T, flag_clear_dict=True, 
            flag_sample_exist=flag_sample_exist,path_sample=path_sample)

    if flag_sample_exist is False:
        for i in range(nsamp):
            out1 = sparse.csc_matrix(np.round(A._IM_dict['pgv'][i].log1p(),decimals=3))
            out2 = sparse.csc_matrix(np.round(A._IM_dict['pga'][i].log1p(),decimals=3))
            sparse.save_npz(im_dir+'/'+im_group+'/'+'pgv_samp_'+str(i)+'.npz',out1)
            sparse.save_npz(im_dir+'/'+im_group+'/'+'pga_samp_'+str(i)+'.npz',out2)

    #####################################################################
    dm_method = []
    ## get rr from HAZUS (FEMA, 2014)
    category = 'rr'
    method = 'hazus_2014_rr'
    return_param = ['rr_pgv']
    flag_pgv = True
    dm_method.append(method[0:method.find('_')])
    A.get_DM(category, method, return_param, im_list, flag_clear_dict=False,
             flag_pgv=flag_pgv, nsamp=nsamp)
#     print(A._DM_dict['rr_pgv']['method1']['output'][0][0:5])
    
    ## get rr from ALA (2001)
    category = 'rr'
    method = 'orourke_2020_rr'
    return_param = ['rr_pgv']
    flag_pgv = True
    dm_method.append(method[0:method.find('_')])
    A.get_DM(category, method, return_param, im_list, flag_clear_dict=False,
             flag_pgv=flag_pgv, nsamp=nsamp)
#     print(A._DM_dict['rr_pgv']['method2']['output'][0][0:5])

    #####################################################################
#     if count % inc1 == 0:
#     print('folder '+str(count)+': '+im_group+'--- %10.6f seconds ---' % (time.time() - time_start))
#     time_start = time.time()

    #####################################################################
#     if count % inc2 == 0 or count == len(groups2run)-1:
#         print('saved files: '+str(range_start)+'_'+str(range_end))
#         for i in range(len(return_param)):
#             count_dm = 0
#             for j in A._DM_dict[return_param[i]]:
#                 save_name = dm_dir+'/'+return_param[i]+'_'+dm_method[count_dm]+'_'+str(range_start)+'_'+str(range_end)+'.txt'
# #                 print(save_name)
#                 np.savetxt(save_name,np.transpose(A._DM_dict[return_param[i]][j]['output']),fmt='%6.4E')
#                 count_dm += 1

10
hazus_2014_rr
[0.00205299 0.00191743 0.0011977  0.00241055 0.00620862 0.00123407
 0.00276242 0.00189023]
 
8
hazus_2014_rr
[1.36438462 3.46218829 0.02037376 0.29553989 0.16962114 0.01882809
 0.01315913 0.02700143]
 
62
hazus_2014_rr
[0.52250274 0.45301465 0.16922593 1.25959311 0.17201149 0.46250276
 1.67916269 0.57020557]
 
10
orourke_2020_rr
[0.0004797  0.00044464 0.00026359 0.00057338 0.0016405  0.0002725
 0.0006671  0.00043763]
 
8
orourke_2020_rr
[0.65634917 1.84707139 0.00614321 0.11995015 0.06472496 0.0056276
 0.0037797  0.00840043]
 
62
orourke_2020_rr
[0.22592809 0.19280027 0.06455742 0.60058173 0.06573922 0.19729222
 0.82662442 0.24895966]
 


In [31]:
rup_marc = [10, 8, 62]

In [32]:
A._RV_dict['rup']['M'][rup_marc]

array([5.95459511, 6.93157737, 7.71964645])

In [33]:
A._RV_dict['rup']['rate'][rup_marc]

array([0.00012348, 0.01114423, 0.00016967])

In [38]:
pgv = A._IM_dict['pgv'][0].toarray()

In [41]:
pgv[:,ind_marc][6]

array([19.36871204,  1.74560102, 17.7276305 , 21.28692091, 19.02537061,
       11.87129966,  6.89318714,  8.74739432])

In [65]:
A._DM_dict

{'rr_pgv': {'method1': {'method': 'hazus_2014_rr',
   'source_param': None,
   'source_method': None,
   'output': array([[0.00117442, 0.00075226, 0.00058834, ..., 0.00026506, 0.00046348,
           0.00080263],
          [0.00042003, 0.00050017, 0.00016908, ..., 0.000579  , 0.00034428,
           0.00032627],
          [0.00048442, 0.00030545, 0.0004575 , ..., 0.00045737, 0.0002526 ,
           0.00038607],
          ...,
          [0.00022358, 0.00036343, 0.00063471, ..., 0.00022467, 0.00024973,
           0.00072461],
          [0.00053118, 0.00031321, 0.00042446, ..., 0.00046433, 0.00105056,
           0.00036599],
          [0.00045225, 0.00022892, 0.00093258, ..., 0.00039517, 0.00060365,
           0.00069788]])},
  'method2': {'method': 'orourke_2020_rr',
   'source_param': None,
   'source_method': None,
   'output': array([[4.78546915e-04, 2.92018252e-04, 2.35861597e-04, ...,
           1.11981796e-04, 1.91499204e-04, 3.61532439e-04],
          [1.57100873e-04, 1.90104963e-04,

In [64]:
rup_marc = [10, 8, 62]
pgv = A._IM_dict['pgv'][5].toarray()
for i in rup_marc:
#     if pgv[i,ind_marc[0]] > 0:
    print(str(i)+': '+str(A._RV_dict['rup']['M'][i])+', '+str(A._RV_dict['rup']['rate'][i]))
    print(pgv[i,ind_marc])

10: 5.95459510870155, 0.00012348449023624983
[3.83074162 3.71618401 3.01485005 4.11409268 6.26450384 3.05519997
 4.37092421 3.69266194]
8: 6.639466161281467, 0.001702372352063113
[ 68.82555079 104.10922     10.62316396  34.87354085  27.24735995
  10.2571108    8.74739432  12.03971891]
62: 7.618024588567435, 0.0001418795360809041
[44.92455769 42.16370586 27.21912671 66.42393004 27.41735358 42.5539326
 75.47776156 46.70327243]
